**Antonio Alejo Module 1 Python for Data Wrangling Basics**

In [1]:
!pip install praw
!pip install asyncpraw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.7/196.7 kB 4.8 MB/s eta 0:00:00


In [3]:
import asyncpraw
import praw
from datetime import datetime

In [4]:
uname = 'tonyalejo33'
upass = 'Isabella1!'
app_id = '01r6wDLQmUWdHwnUaExo9w'
app_secret = 'hD_FJs6KSanhWiP-CKFqmPZInvNlSg'

In [5]:
reddit = asyncpraw.Reddit(user_agent = 'Heat in PHX (by /u/tonyalejo33)',
                     client_id = app_id, client_secret = app_secret,
                     username = uname, password = upass, check_for_async = False)

In [6]:
subreddit = await reddit.subreddit('Phoenix', fetch = True)

In [7]:
async def is_recent(submission):
    current_time = datetime.utcnow()
    submission_time = datetime.utcfromtimestamp(submission.created_utc)
    time_difference_seconds = (current_time - submission_time).total_seconds()
    return time_difference_seconds < (72 * 3600)

In [8]:
def contains_keywords(text):
    keywords = ['cheese', 'pickles', 'buns', 'meat', 'sesame']
    return any(keyword.lower() in text.lower() for keyword in keywords)

In [9]:
async for submission in subreddit.hot(limit = 10):
  if await is_recent(submission) and contains_keywords(submission.title):
        print(submission.title)

In [10]:
topiclist = []
async for submission in subreddit.new(limit = 7):
  topiclist.append(submission.id)

In [11]:
topiclist

['1b2tly3', '1b2t1zd', '1b2sovy', '1b2qhf2', '1b2pc84', '1b2nxtp', '1b2nnk2']

In [12]:
from time import sleep

In [14]:
with open('subfile.csv', 'w') as subfile, open('comfile.csv', 'w') as comfile:
  for eachtopic in topiclist:

    sleep(2)
    submission = await reddit.submission(eachtopic)

    lineout = eachtopic + ',"'
    lineout += submission.title + '",'
    lineout += submission.author.name + ','
    lineout += str(submission.created_utc) + ','
    lineout += str(submission.score) + '\n'

    print(lineout)
    subfile.write(lineout)

    submission.comments.replace_more(limit = None)
    commentlist = submission.comments.list()

    for eachcomment in commentlist:

      sleep(1)

      if isinstance(eachcomment, asyncpraw.models.MoreComments):
            continue

      if await is_recent(eachcomment) and contains_keywords(eachcomment.body):
            lineout = eachcomment.id + ',"'
            lineout += eachcomment.body.replace('\n', '/') + '",'

      if not eachcomment.author:
        name = 'NoName'
      else:
        name = eachcomment.author.name
      lineout += name + ','


      lineout += str(eachcomment.created_utc) + ','
      lineout += str(eachcomment.score) + ','
      lineout += eachcomment.parent_id + ','
      lineout += eachtopic + '\n'

      #print(lineout)
      comfile.write(lineout)

1b2tly3,"I’m staying 10 min north of Phoenix Art Museum and here for 4 days before surgery, what should I do? ",GoodMoriningVeitnam,1709187910.0,2



<ipython-input-14-25b9b59913ff>:16: RuntimeWarning: coroutine 'CommentForest.replace_more' was never awaited
  submission.comments.replace_more(limit = None)


1b2t1zd,"Need new fire alarm system installed, any recommendations?",tbrmax,1709186022.0,0

1b2sovy,"Best place to shop antique stores for stained glass?",rallruse,1709184806.0,2

1b2qhf2,"Easter brunch recommendation",Ohparrothead,1709177771.0,0

1b2pc84,"Thinking Of Moving To Phoenix: Tell Me Everything About The Phoenix Area",RegainingLife,1709174464.0,0

1b2nxtp,"Arizona is one of the states for the IRS’s new “Direct File” program. ",haikufive,1709170518.0,14

1b2nnk2,"Former top Phoenix prosecutor arrested on felony harassment charge (September 2023)",RNAprimer,1709169695.0,21

